# Lesson 5: Distributed Scaling

In lesson 4 you've learned how to be as efficient as possible on a single CPU.

Now we want to use multiple CPUs at the same time (horizontal scaling).

![image](https://raw.githubusercontent.com/nsmith-/2025-09-15-hsf-india-tutorial-chandigarh/cda03896f0f7d4d195db3fedf00a8754f6774214/img/horizontal-and-vertical-scaling.svg)

## Scaling on a single machine

A single computer may have multiple cores available that we can use to concurrently run software.

In python the two ways to parallelize programs are: **_multi-processing_** and **_multi-threading_**.

**Multi-processing** is essentially about running multiple OS processes in parallel, each with its own Python interpreter, memory space, etc.

**Multi-threading**, however, uses a shared memory space and a common Python interpreter to spawn multiple threads that can run tasks concurrently. 
_Caution_: In Python multi-threading is in fact not running programs in parallel because of the Global Interpreter Lock (GIL), only since python 3.14 (free-threaded) this is possible.

In [1]:
import concurrent.futures

In [2]:
%%writefile quadratic_formula_scaling.py

# writing to a separate file, because multiprocessing is buggy in notebooks (see: https://bugs.python.org/issue25053)
import numpy as np
import time

a = np.random.uniform(5, 10, 1_000_000)
b = np.random.uniform(10, 20, 1_000_000)
c = np.random.uniform(-0.1, 0.1, 1_000_000)

def quadratic_formula(task_id):
    time.sleep(task_id / 5.0)  # simulate some task_id-dependent workload
    return (-b + np.sqrt(b**2 - 4*a*c)), task_id  # return task_id for identification


# setup 10 tasks
task_ids = range(10)

Overwriting quadratic_formula_scaling.py


### Python for-loop example

In [3]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids

for task_id in task_ids:
    output, _ = quadratic_formula(task_id)
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 0 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 1 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 2 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 3 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 4 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 5 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 6 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 7 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 8 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 9 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
9.27 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Multi-threading example

In [4]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(quadratic_formula, task_id) for task_id in task_ids]

for future in concurrent.futures.as_completed(futures):
    output, task_id = future.result()
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 9 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 6 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 3 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 5 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 1 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 4 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 7 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 8 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 2 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
Task 0 completed with output [ 0.01238954 -0.0634219  -0.04013361  0.03417539 -0.00302836]...
1.83 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Multi-processing example

In [5]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids

with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(quadratic_formula, task_id) for task_id in task_ids]

for future in concurrent.futures.as_completed(futures):
    output, task_id = future.result()
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 1 completed with output [-0.09104369 -0.01559355 -0.03353829 -0.02387317  0.10139965]...
Task 8 completed with output [ 0.03137814  0.05093763 -0.00684429 -0.01421032  0.03578165]...
Task 3 completed with output [-0.0793137  -0.14244971  0.0731121  -0.00754798  0.06479027]...
Task 9 completed with output [-0.03610273  0.0555374  -0.09098693 -0.0737754   0.04332356]...
Task 4 completed with output [-0.05006251  0.03575988 -0.00529513  0.08127047  0.09364542]...
Task 2 completed with output [ 0.06590623 -0.00322406  0.06861671  0.05795695 -0.13101162]...
Task 7 completed with output [ 0.04928703 -0.04993125 -0.05959507 -0.03575024 -0.11078752]...
Task 0 completed with output [-0.03621002  0.11533992  0.03860327 -0.09948181 -0.02707915]...
Task 5 completed with output [ 0.0596364   0.02650107 -0.02362265  0.00557503 -0.01815983]...
Task 6 completed with output [-0.10368085 -0.0861      0.10750636  0.02624262 -0.1164529 ]...
2.08 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

Multi-threading and multi-processing can speed-up your programs by concurrently executing multiple tasks across multiple threads or processes.

Here, we can see in addition the effect of shared (**multi-threading**) vs individual (**multi-processing**) memory space: 
- **multi-threading**: the NumPy random initialization runs _once_ and the memory is shared between all threads, which is why all outputs are the _same_ (`[ 0.00466771  0.00414385  0.00415603 -0.00520032 -0.00212164]...`)
- **multi-processing**: the NumPy random initialization runs in _each_ process again. There's no shared memory, every process has it's own set of `a`, `b`, and `c` arrays, which is why all outputs are _different_. Initializing the arrays in each process adds an additional runtime overhead, which is why here the multi-processing case is slightly slower.

## Scaling on multiple machines

## Dask

There are different ways to scale to multiple machines in a computing cluster. 

The most classic ways are independent of Python and let you submit any type of program 'job' to a cluster, e.g. using HTCondor or Slurm.

In Python, one solution emerged that allows to submit jobs/tasks to a cluster of workers from a Python program: **Dask**.

### Dask example

In [6]:
import dask.distributed

from quadratic_formula_scaling import quadratic_formula, task_ids

In [7]:
%%timeit -n 1 -r 1

with (
    # multi-threading cluster
    dask.distributed.LocalCluster(n_workers=1, threads_per_worker=10) as cluster,
    # multi-processing cluster
    # dask.distributed.LocalCluster(n_workers=10, threads_per_worker=1) as cluster,
    # mixed cluster
    # dask.distributed.LocalCluster(n_workers=2, threads_per_worker=5) as cluster,
    # ---
    # connect to a Dask cluster
    dask.distributed.Client(cluster) as client,
):

     futures = client.map(quadratic_formula, task_ids)

     for future in dask.distributed.as_completed(futures):
         output, task_id = future.result()
         print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 0 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 1 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 2 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 3 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 4 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 5 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 6 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 7 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 8 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
Task 9 completed with output [-0.030239   -0.03803548  0.07324641  0.04012371  0.11241015]...
2.52 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Dask in practice

Dask abstracts the cluster from the user. If there's a dask-scheduler running somewhere, we can connect to it using:

```python
with dask.distributed.Client(address='127.0.0.1:8786') as client: # some IP+port address
    ...
```

This is great, as scientist never have to worry about the cluster infrastructure, with a single lince of code you can connect and run your analysis distributed on multiple workers and optionally using multiple threads per worker.

One cluster in HEP that's commonly used for analysis is: https://coffea.casa/.

### Dask is much more than a distributive interface

Dask allows to define compute graphs of complex workflows. These graphs can then be scheduled automatically by the dask-scheduler such that the most optimal execution of tasks is achieved.

In [8]:
import dask

In [9]:
@dask.delayed  # mark function as delayed/task
def increment(i):
    return i + 1

@dask.delayed  # mark function as delayed/task
def add(a, b):
    return a + b


# define compute graph
a, b = 1, 12
c = increment(a)
d = increment(b)
output = add(c, d)

# visualize compute graph
try:
    output.visualize(rankdir="LR", filename="compute_graph_simple.svg")  # left to right
except Exception as e:
    ... # install graphviz and python-graphviz to visualize the graph

# compute the result
print("Result:", output.compute())

Result: 15


![image](compute_graph_simple.svg)

In [10]:
import numpy as np
import dask.array as da

# use dask arrays instead of Numpy arrays for automatic graph construction
a = da.random.uniform(5, 10, 100_000)
b = da.random.uniform(10, 20, 100_000)
c = da.random.uniform(-0.1, 0.1, 100_000)

def quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

# define compute graph
output = quadratic_formula(a, b, c)

# visualize compute graph
try:
    output.visualize(rankdir="LR", filename="compute_graph_quadratic_formula.svg")
    output.visualize(rankdir="LR", filename="compute_graph_quadratic_formula__optimized.svg", optimize_graph=True)
except Exception as e:
    ... # install graphviz and python-graphviz to visualize the graph


# compute the result
unoptimized_result = dask.compute(output, optimize_graph=False)[0]
print("Result (unoptimized):",unoptimized_result)

optimized_result = dask.compute(output, optimize_graph=True)[0]
print("Result (optimized):",optimized_result)

assert np.allclose(unoptimized_result, optimized_result)


Result (unoptimized): [-0.00450413  0.00084627  0.00455701 ... -0.00321069  0.00398654
 -0.00650789]
Result (optimized): [-0.00450413  0.00084627  0.00455701 ... -0.00321069  0.00398654
 -0.00650789]


#### Compute Graph

![image](compute_graph_quadratic_formula.svg)

#### Compute Graph _optimized_

![image](compute_graph_quadratic_formula__optimized.svg)

In [11]:
def fun(a, b, c):
    return np.mean((-b + np.sqrt(b**2 - 4*a*c)) / (2*a))

In [12]:
%%timeit -n 1 -r 1

a = np.random.uniform(5, 10, 200_000_000)
b = np.random.uniform(10, 20, 200_000_000)
c = np.random.uniform(-0.1, 0.1, 200_000_000)

output = fun(a, b, c)
print("Result:", output)

Result: -9.743639927098622e-06
3.74 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
# A too slow problem for a single machine... (200M points)

# use dask arrays instead of Numpy arrays for automatic graph construction
da_a = da.random.uniform(5, 10, 200_000_000)
da_b = da.random.uniform(10, 20, 200_000_000)
da_c = da.random.uniform(-0.1, 0.1, 200_000_000)

# define compute graph
output = fun(da_a, da_b, da_c)

print(output.dask)

HighLevelGraph with 15 layers.
 0. uniform-a6cdf9fc8313e4da47fb3d7618baccf6
 1. uniform-a6637a900161fb17a0b7fc53cc92c973
 2. mul-035a54ca02fead41ece9aa0677258373
 3. mul-36562f199e8173d6211eff6a859c5802
 4. mul-198e1833b751db9a1cfd652ec8fd372f
 5. uniform-29f25fee0b7624aecf4377df7449b247
 6. pow-f1248be5c6f02766e24fcc97dd6bff23
 7. sub-b0a40d65e8dfa04064c2c2d8c7506749
 8. sqrt-1d317540828f1db92dd1b7b28e5fb8c3
 9. neg-d72e1a8d61054cb5ece412a178700a88
 10. add-c167ff191e5a0657b0eeaba73eb4e883
 11. truediv-d8d7aa71496c7b1f01cb72f2c17d93c6
 12. mean_chunk-5460fbac0c514fbf7ab3358aed1f4525
 13. mean_combine-partial-3f3a31abc50aa0b64e4731a9c7c62a18
 14. mean_agg-aggregate-561ce9ae03476b6f02e45bba0b44d689



In [14]:
%%timeit -n 1 -r 1

with (
    dask.distributed.LocalCluster(n_workers=1, threads_per_worker=10) as cluster,
    # ---
    # connect to a Dask cluster
    dask.distributed.Client(cluster) as client,
):
    # compute the result
    print("Result (unoptimized):", client.compute(output, optimize_graph=False).result())

Result (unoptimized): -9.183300095630478e-06
1.51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit -n 1 -r 1

with (
    dask.distributed.LocalCluster(n_workers=1, threads_per_worker=10) as cluster,
    # ---
    # connect to a Dask cluster
    dask.distributed.Client(cluster) as client,
):
    # compute the result
    print("Result (optimized):", client.compute(output, optimize_graph=True).result())

Result (optimized): -9.183300095630478e-06
1.53 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
